## Grundlagen Maschineller Lernverfahren | ML_INF19A | 2021
**Datum: 14.10.2021**

# Naive Bayes (Gaussian)

In [ ]:
# Lade Bibliotheken

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


## Daten erzeugen

In [ ]:
# Lade Datensatz (IRIS)

iris = load_iris()

In [ ]:
print(iris.DESCR)

# Sepal = Kelchblatt
# Petal = Blütenblatt

In [ ]:
# Daten anzeigen lassen

iris.data

In [ ]:
# Separiere Werte in einzelne Variablen

X = iris.data[:,2:4] # Nur petal length / width -> Blütenblatt
y = iris.target

In [ ]:
# Trainingsdaten
X

In [ ]:
# Trainingsdaten

y

## Datensätze aufsplitten 

In [ ]:
# Training + Validierung

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## NaiveBayes Modell

In [ ]:
# Lade Bibliothek

from sklearn.naive_bayes import GaussianNB

In [ ]:
# Erstelle NaiveBayes Instanz

nb_clf = GaussianNB()

In [ ]:
# Trainiere
nb_clf.fit(X_train, y_train)

# Mache Vorhersage
y_prediction = nb_clf.predict(X_test)

In [ ]:
# Analysiere

print("Anzahl der falsch zugeordneten Klassen: %d (bei %d Datensätze)" % ((y_test != y_prediction).sum(), X_test.shape[0]))

In [ ]:
# Berechne Accuracy Score

from sklearn.metrics import accuracy_score
my_accuracy_score = accuracy_score(y_test, y_prediction)
print("Accuracy: %0.2f  [%s]" % (my_accuracy_score, nb_clf))

# Vergleich: SGDClassifier

In [ ]:
# Lade Bibliothek
from sklearn.linear_model import SGDClassifier

# Erstelle Instanz
sgd_class = SGDClassifier()

# Training
sgd_class.fit(X_train, y_train)

# Vorhersage
y_prediction_sgd = sgd_class.predict(X_test)

# Analysiere
print("Anzahl der falsch zugeordneten Klassen: %d (bei %d Datensätze)" % ((y_test != y_prediction_sgd).sum(), X_test.shape[0]))

# Berechne Accuracy Score
my_accuracy_score_sgd = accuracy_score(y_test, y_prediction_sgd)
print("Accuracy: %0.2f  [%s]" % (my_accuracy_score_sgd, sgd_class))


# Hinweis: Die Daten wurden nicht skaliert!

# Zusatz: Hyperparametersuche

Hier am Beispiel der Linearen Regression

Ressource: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection

In [ ]:
# Lade notwendige Bibliotheken

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Pseudo Zufallszahlen

np.random.seed(2020)

In [ ]:
# Daten erstellen (vgl. Übung Optimierung)

X = 2*np.random.rand(100,1)
y = 4+3*X + np.random.rand(100,1)

# Erstelle zwei x-Werte zum Testen
X_unknown = np.array([[0],[2]])

In [ ]:
# Lade SGD Lin Reg Modell

from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor() # Standardwerte! -> Verweis auf Dokumentation


In [ ]:
# Erstelle Parametergrid -> Prüfe welche Parameter der Optimierer hat
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html


# Hinweis: Größe des Suchraums beeinflußt die benötigte Zeit für die Suche!
parameters = {'loss':('squared_loss', 'huber'), 
              'penalty': ('l2', 'l1', 'elasticnet'),
              'alpha': [1e-1, 1e-2, 1e-3, 1e-4, 5e-4, 1e-5],
              'max_iter': np.arange(start = 700, stop = 1500, step = 100),
              'learning_rate': ['constant', 'optimal', 'invscaling']}
            

#### Vollständige Suche im Parameterraum

In [ ]:
# Lade Bibliothek
from sklearn.model_selection import GridSearchCV

# Erstelle Instanz (Modell + Parametergrid)
gridsearch = GridSearchCV(sgd_reg, parameters)

# Starte die Suche (analog zu "normalem" Training)
gridsearch.fit(X,y.ravel())

In [ ]:
# Prüfe die besten Parameter
gridsearch.best_params_

In [ ]:
# Das beste, vollständig konfigurierte Modell erhalten
best_model_gs = gridsearch.best_estimator_

In [ ]:
# Vorhersage mit dem besten Modell machen
best_model_gs.predict(X_unknown)

# Anwendung einer Preprocessing Pipeline

In [ ]:
# Lade Bibliotheken

# Pipelines
from sklearn.pipeline import make_pipeline # https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html

# Fülle fehlende Datenwerte automatisiert auf
from sklearn.impute import SimpleImputer # https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

# Skaliere Daten
from sklearn.preprocessing import MinMaxScaler # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html



In [ ]:
# Erstelle eine Pipeline namens "preprocessing_pipe" um die Daten vorzuverarbeiten
#     Teil 1: SimpleImputer
#     Teil 2: MinMaxScaler

preprocessing_pipe = make_pipeline(
    SimpleImputer(missing_values=np.NaN, strategy='mean'),
    MinMaxScaler(feature_range=(-1,1))
)

# Ein Aufruf für Ergänzung und Skalierung
X_pp = preprocessing_pipe.fit_transform(X)

In [ ]:
# X.shape

In [ ]:
# Dimensionen auf Gleichheit prüfen
assert X_pp.shape[0] == X.shape[0]
assert X_pp.shape[1] == X.shape[1]

In [ ]:
# Zeige X an
X

In [ ]:
# Zeige X_pp (nach Preprocessing Pipeline) an
X_pp

In [ ]:
# Starte die Suche (diesmal mit den transformierten X Werten X_pp)
gridsearch.fit(X_pp, y.ravel())

In [ ]:
# Prüfe die besten Parameter
gridsearch.best_params_